In [ ]:
# ============================================================================
# ИМПОРТ БИБЛИОТЕК
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import warnings
from collections import defaultdict, deque
import logging
import pickle
import json
from pathlib import Path

from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Настройка логирования
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Настройка отображения pandas
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Подавление предупреждений
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

logger.info("Библиотеки успешно загружены")

In [ ]:
# ============================================================================
# КОНФИГУРАЦИЯ ПАРАМЕТРОВ
# ============================================================================

class ModelConfig:
    
    # Пути к данным
    DATA_DIR = Path("data")
    TRAIN_PATH = DATA_DIR / "train_data.parquet"
    
    # Директория для моделей
    MODEL_DIR = Path("models")
    MODEL_VERSION = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Временные границы разделения данных
    TRAIN_CUTOFF = "2025-01-31"       # Конец обучающей выборки
    VALIDATION_CUTOFF = "2025-03-31"  # Конец валидационной выборки
    # OOT (Out-of-Time) тестовая выборка: после VALIDATION_CUTOFF
    
    # Прогнозирование
    FORECAST_START = "2025-10-31"
    HORIZON_MONTHS = 6
    MIN_SAMPLES_PER_SEGMENT = 1000
    
    # Признаки модели
    CATEGORICAL_FEATURES = [
        'QUALITY_CODE',     
        'SUBJECT_KIND_ID',   
        'EC_SECTOR_ID',      # Используем как категориальную фичу вместо разделения по сегментам
        'SEGMENT_ID'         # Оставляем SEGMENT_ID как категориальную фичу
    ]
    
    BASE_FEATURES = [
        # Текущие и лаговые значения маржи
        'MARGIN', 'MARGIN_LAG1', 'MARGIN_LAG2', 'MARGIN_LAG3',
        # Скользящие средние
        'MARGIN_AVG_1M_LAG', 'MARGIN_AVG_2M_LAG', 'MARGIN_AVG_3M_LAG',
        'MARGIN_AVG_6M_LAG', 'MARGIN_AVG_12M_LAG',
        # Волатильность и тренд
        'MARGIN_STDDEV_12M_LAG', 'MARGIN_GROWTH_RATE_3M',
        # Временные признаки
        'MONTH_OF_YEAR', 'QUARTER_OF_YEAR', 'TENURE_MONTHS'
    ]
    
    # Гиперпараметры CatBoost
    CATBOOST_PARAMS = {
        'iterations': 1600,           
        'depth': 4,                  
        'learning_rate': 0.05,        
        'l2_leaf_reg': 3,             
        'random_seed': 42,           
        'loss_function': 'RMSE',     
        'verbose': 100,               
        'early_stopping_rounds': 50   
    }
    
    @classmethod
    def ensure_directories(cls):
        cls.MODEL_DIR.mkdir(parents=True, exist_ok=True)
        (cls.MODEL_DIR / cls.MODEL_VERSION).mkdir(parents=True, exist_ok=True)

# Инициализация конфигурации
ModelConfig.ensure_directories()

logger.info(f"Версия модели: {ModelConfig.MODEL_VERSION}")
logger.info(f"Обучающая выборка: до {ModelConfig.TRAIN_CUTOFF}")
logger.info(f"Валидационная выборка: {ModelConfig.TRAIN_CUTOFF} - {ModelConfig.VALIDATION_CUTOFF}")
logger.info(f"OOT тестовая выборка: после {ModelConfig.VALIDATION_CUTOFF}")

In [ ]:
# ============================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ============================================================================

def load_parquet_data(file_path: Path) -> pd.DataFrame:
    if not file_path.exists():
        logger.warning(f"Файл не найден: {file_path}")
        return pd.DataFrame()
    
    try:
        df = pd.read_parquet(file_path)
        logger.info(f"Загружено {len(df):,} записей из {file_path.name}")
        return df
    except Exception as e:
        logger.error(f"Ошибка загрузки {file_path}: {e}")
        return pd.DataFrame()


def preprocess_categorical_features(df: pd.DataFrame, 
                                   categorical_cols: list) -> pd.DataFrame:
    df_processed = df.copy()
    
    for col in categorical_cols:
        if col in df_processed.columns:
            # Заполнение пропусков
            df_processed[col] = df_processed[col].fillna('UNKNOWN')
            # Приведение к строке
            df_processed[col] = df_processed[col].astype(str)
            # Удаление .0 из строковых представлений чисел
            df_processed[col] = df_processed[col].str.replace('.0', '', regex=False)
    
    return df_processed


def transform_target(y: pd.Series) -> pd.Series:
    return np.sign(y) * np.log1p(np.abs(y))


logger.info("Вспомогательные функции загружены")

In [ ]:
# ============================================================================
# ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ
# ============================================================================

logger.info("Начало загрузки данных...")

# Загрузка данных из Parquet
df_train = load_parquet_data(ModelConfig.TRAIN_PATH)

# Проверка успешности загрузки
if df_train.empty:
    logger.error("Ошибка: данные не загружены!")
    logger.info("Надо прогнать 'data_loader.ipynb'")
else:
    logger.info(f"Обучающая выборка: {len(df_train):,} записей")
    logger.info(f"Уникальных клиентов: {df_train['CLIENT_ID'].nunique():,}")

In [ ]:
# ============================================================================
# EXPLORATORY DATA ANALYSIS
# ============================================================================

logger.info("Начало разведочного анализа данных...")

print("\n" + "="*80)
print("ОБЩАЯ ИНФОРМАЦИЯ О ДАННЫХ")
print("="*80)
print(f"Размер данных: {df_train.shape[0]:,} строк x {df_train.shape[1]} столбцов")
print(f"Период данных: {df_train['MONTH_END'].min()} - {df_train['MONTH_END'].max()}")
print(f"Уникальных клиентов: {df_train['CLIENT_ID'].nunique():,}")

print("\n" + "="*80)
print("ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ")
print("="*80)
missing = df_train.isnull().sum()
missing_pct = 100 * missing / len(df_train)
missing_df = pd.DataFrame({
    'Столбец': missing.index,
    'Пропусков': missing.values,
    'Процент': missing_pct.values
})
missing_df = missing_df[missing_df['Пропусков'] > 0].sort_values('Пропусков', ascending=False)
if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("Пропущенных значений не обнаружено")

print("\n" + "="*80)
print("РАСПРЕДЕЛЕНИЕ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ (TARGET_NEXT_MARGIN)")
print("="*80)
target_stats = df_train['TARGET_NEXT_MARGIN'].describe()
print(target_stats)

print("\n" + "="*80)
print("РАСПРЕДЕЛЕНИЕ ПО СЕГМЕНТАМ")
print("="*80)
segment_counts = df_train['SEGMENT_ID'].value_counts()
for segment, count in segment_counts.items():
    pct = 100 * count / len(df_train)
    print(f"{segment}: {count:,} ({pct:.1f}%)")

print("\n" + "="*80)
print("СТАТИСТИКА МАРЖИ ПО СЕГМЕНТАМ")
print("="*80)
margin_by_segment = df_train.groupby('SEGMENT_ID')['TARGET_NEXT_MARGIN'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max')
])
print(margin_by_segment)

# Визуализация распределения целевой переменной
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Гистограмма
axes[0].hist(df_train['TARGET_NEXT_MARGIN'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('TARGET_NEXT_MARGIN')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение целевой переменной')
axes[0].grid(alpha=0.3)

# Box plot по сегментам
df_train.boxplot(column='TARGET_NEXT_MARGIN', by='SEGMENT_ID', ax=axes[1])
axes[1].set_xlabel('Сегмент')
axes[1].set_ylabel('TARGET_NEXT_MARGIN')
axes[1].set_title('Распределение маржи по сегментам')
plt.suptitle('')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# ПРОВЕРКА ДАННЫХ
# ============================================================================

logger.info("Проверка загруженных данных...")

print("\n" + "="*80)
print("ИНФОРМАЦИЯ О СЕГМЕНТАХ")
print("="*80)
print(df_train['SEGMENT_ID'].value_counts().sort_index())

print("\n" + "="*80)
print("ИНФОРМАЦИЯ О EC_SECTOR_ID (BUS_SECTOR_ID)")
print("="*80)
print(df_train['EC_SECTOR_ID'].value_counts().sort_index())

# Детальная статистика
print("\n" + "="*80)
print("ОБЩАЯ СТАТИСТИКА")
print("="*80)

n_records = len(df_train)
n_clients = df_train['CLIENT_ID'].nunique()
avg_margin = df_train['TARGET_NEXT_MARGIN'].mean()

print(f"\nВсего данных:")
print(f"  Записей: {n_records:>12,}")
print(f"  Клиентов: {n_clients:>11,}")
print(f"  Средняя маржа: {avg_margin:>12,.2f}")

print(f"\nСегменты (SEGMENT_ID):")
for segment_id in sorted(df_train['SEGMENT_ID'].unique()):
    segment_data = df_train[df_train['SEGMENT_ID'] == segment_id]
    n_seg_records = len(segment_data)
    n_seg_clients = segment_data['CLIENT_ID'].nunique()
    pct_records = 100 * n_seg_records / n_records
    
    print(f"  {segment_id}: {n_seg_records:,} записей ({pct_records:.1f}%), {n_seg_clients:,} клиентов")

print(f"\nБизнес-секторы (EC_SECTOR_ID):")
for sector_id in sorted(df_train['EC_SECTOR_ID'].unique()):
    sector_data = df_train[df_train['EC_SECTOR_ID'] == sector_id]
    n_sec_records = len(sector_data)
    pct_records = 100 * n_sec_records / n_records
    
    print(f"  {sector_id}: {n_sec_records:,} записей ({pct_records:.1f}%)")

logger.info("Проверка данных завершена")

In [ ]:
# ============================================================================
# ФОРМИРОВАНИЕ ПРИЗНАКОВОГО ПРОСТРАНСТВА
# ============================================================================

logger.info("Подготовка признаков...")

# Предобработка категориальных признаков
df_train_processed = preprocess_categorical_features(
    df_train, ModelConfig.CATEGORICAL_FEATURES
)

# Полный список признаков (базовые + категориальные)
all_features = ModelConfig.BASE_FEATURES + ModelConfig.CATEGORICAL_FEATURES

# Фильтрация доступных признаков
available_features = [
    feature for feature in all_features 
    if feature in df_train_processed.columns
]

# Заполнение пропусков в числовых признаках
numeric_features = [
    feature for feature in available_features 
    if feature not in ModelConfig.CATEGORICAL_FEATURES
]

for feature in numeric_features:
    df_train_processed[feature] = df_train_processed[feature].fillna(0.0)

logger.info(f"Всего признаков: {len(available_features)}")
logger.info(f"  Числовые: {len(numeric_features)}")
logger.info(f"  Категориальные: {len(ModelConfig.CATEGORICAL_FEATURES)}")

print(f"\nСписок признаков модели:")
print(f"  Всего: {len(available_features)}")
print(f"  Числовые ({len(numeric_features)}): {', '.join(numeric_features[:5])}...")
print(f"  Категориальные ({len(ModelConfig.CATEGORICAL_FEATURES)}): {', '.join(ModelConfig.CATEGORICAL_FEATURES)}")

In [ ]:
# ============================================================================
# КЛАСС ЕДИНОЙ МОДЕЛИ CLTV (БЕЗ СЕГМЕНТАЦИИ)
# ============================================================================

class CLTVModel:
    """
    Единая модель CLTV для всех клиентов сегмента SMALL.
    Использует EC_SECTOR_ID и SEGMENT_ID как категориальные признаки вместо разделения.
    """
    
    def __init__(self, model_params: dict):
        self.model = None
        self.metrics = {}
        self.feature_importance = None
        self.model_params = model_params
        
    def train(
        self,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_val: pd.DataFrame,
        y_val: pd.Series,
        X_test: pd.DataFrame,
        y_test: pd.Series,
        categorical_features: list = None
    ) -> dict:
        """
        Обучение единой модели на всех данных.
        """
        logger.info(f"\nОбучение единой модели CLTV")
        logger.info(f"  Train: {len(X_train):,} записей")
        logger.info(f"  Validation: {len(X_val):,} записей")
        logger.info(f"  OOT Test: {len(X_test):,} записей")
        
        # Определение индексов категориальных признаков
        categorical_indices = []
        if categorical_features:
            feature_names = X_train.columns.tolist()
            categorical_indices = [
                feature_names.index(feature) 
                for feature in categorical_features 
                if feature in feature_names
            ]
            logger.info(f"  Категориальные признаки: {categorical_features}")
        
        # Создание Pool объектов для CatBoost
        train_pool = Pool(X_train, y_train, cat_features=categorical_indices)
        val_pool = Pool(X_val, y_val, cat_features=categorical_indices)
        test_pool = Pool(X_test, y_test, cat_features=categorical_indices)
        
        # Обучение модели
        self.model = CatBoostRegressor(**self.model_params)
        self.model.fit(
            train_pool, 
            eval_set=val_pool, 
            use_best_model=True
        )
        
        # Расчет метрик на всех выборках
        predictions = {
            'train': self.model.predict(X_train),
            'val': self.model.predict(X_val),
            'test': self.model.predict(X_test)
        }
        
        self.metrics = {
            # Обучающая выборка
            'train_r2': r2_score(y_train, predictions['train']),
            'train_rmse': np.sqrt(mean_squared_error(y_train, predictions['train'])),
            'train_mae': mean_absolute_error(y_train, predictions['train']),
            'train_samples': len(X_train),
            # Валидационная выборка
            'val_r2': r2_score(y_val, predictions['val']),
            'val_rmse': np.sqrt(mean_squared_error(y_val, predictions['val'])),
            'val_mae': mean_absolute_error(y_val, predictions['val']),
            'val_samples': len(X_val),
            # OOT тестовая выборка
            'test_r2': r2_score(y_test, predictions['test']),
            'test_rmse': np.sqrt(mean_squared_error(y_test, predictions['test'])),
            'test_mae': mean_absolute_error(y_test, predictions['test']),
            'test_samples': len(X_test),
        }
        
        # Feature importance
        self.feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': self.model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # Вывод результатов
        logger.info("Результаты обучения:")
        logger.info(f"  Train R²: {self.metrics['train_r2']:.4f}")
        logger.info(f"  Val R²:   {self.metrics['val_r2']:.4f}")
        logger.info(f"  Test R²:  {self.metrics['test_r2']:.4f} (OOT)")
        
        return self.metrics
    
    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """
        Предсказание на новых данных.
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Вызовите метод train() сначала.")
        
        return self.model.predict(X)


logger.info("Класс CLTVModel определен")

In [ ]:
# ============================================================================
# ПОДГОТОВКА ОБУЧАЮЩИХ ДАННЫХ
# Разделение на Train / Validation / OOT Test
# ============================================================================

logger.info("Формирование обучающих выборок...")

# Стабилизирующая трансформация целевой переменной
df_train_processed['target_transformed'] = transform_target(
    df_train_processed['TARGET_NEXT_MARGIN']
)

# Временное разделение данных
train_end_date = pd.to_datetime(ModelConfig.TRAIN_CUTOFF)
val_end_date = pd.to_datetime(ModelConfig.VALIDATION_CUTOFF)

month_end_dates = pd.to_datetime(df_train_processed['MONTH_END'])

train_mask = month_end_dates <= train_end_date
val_mask = (month_end_dates > train_end_date) & (month_end_dates <= val_end_date)
test_mask = month_end_dates > val_end_date

# Статистика разделения
print("\n" + "="*80)
print("РАЗДЕЛЕНИЕ ДАННЫХ")
print("="*80)
print(f"Train:      {train_mask.sum():>10,} записей ({100*train_mask.sum()/len(df_train_processed):>5.1f}%)")
print(f"Validation: {val_mask.sum():>10,} записей ({100*val_mask.sum()/len(df_train_processed):>5.1f}%)")
print(f"OOT Test:   {test_mask.sum():>10,} записей ({100*test_mask.sum()/len(df_train_processed):>5.1f}%)")

# Подготовка единого датасета (без разделения по сегментам)
X_train = df_train_processed.loc[train_mask, available_features]
y_train = df_train_processed.loc[train_mask, 'target_transformed']

X_val = df_train_processed.loc[val_mask, available_features]
y_val = df_train_processed.loc[val_mask, 'target_transformed']

X_test = df_train_processed.loc[test_mask, available_features]
y_test = df_train_processed.loc[test_mask, 'target_transformed']

print(f"\n{'='*80}")
print("РАЗМЕРЫ ВЫБОРОК")
print(f"{'='*80}")
print(f"X_train: {X_train.shape}")
print(f"X_val:   {X_val.shape}")
print(f"X_test:  {X_test.shape}")

logger.info("Данные подготовлены для обучения единой модели")

In [ ]:
# ============================================================================
# ОБУЧЕНИЕ МОДЕЛИ
# ============================================================================

logger.info("\nНачало обучения модели...")

print("\n" + "="*80)
print("ОБУЧЕНИЕ ЕДИНОЙ МОДЕЛИ CLTV")
print("="*80)

# Инициализация модели
cltv_model = CLTVModel(model_params=ModelConfig.CATBOOST_PARAMS)

# Обучение модели
metrics = cltv_model.train(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    categorical_features=ModelConfig.CATEGORICAL_FEATURES
)

# Вывод топ-20 важных признаков
print(f"\n{'='*80}")
print("ТОП-20 ВАЖНЫХ ПРИЗНАКОВ")
print(f"{'='*80}")
for idx, row in cltv_model.feature_importance.head(20).iterrows():
    print(f"  {row['feature']:30s}: {row['importance']:6.2f}")

print("\n" + "="*80)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО")
print("="*80)

logger.info("Модель успешно обучена")

In [ ]:
# ============================================================================
# СВОДКА РЕЗУЛЬТАТОВ
# ============================================================================

print("\n" + "="*80)
print("ИТОГОВЫЕ МЕТРИКИ КАЧЕСТВА МОДЕЛИ")
print("="*80)

# Формирование таблицы метрик
summary_data = {
    'Метрика': ['R²', 'RMSE', 'MAE', 'Количество'],
    'Train': [
        f"{metrics['train_r2']:.4f}",
        f"{metrics['train_rmse']:.2f}",
        f"{metrics['train_mae']:.2f}",
        f"{metrics['train_samples']:,}"
    ],
    'Validation': [
        f"{metrics['val_r2']:.4f}",
        f"{metrics['val_rmse']:.2f}",
        f"{metrics['val_mae']:.2f}",
        f"{metrics['val_samples']:,}"
    ],
    'Test (OOT)': [
        f"{metrics['test_r2']:.4f}",
        f"{metrics['test_rmse']:.2f}",
        f"{metrics['test_mae']:.2f}",
        f"{metrics['test_samples']:,}"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + summary_df.to_string(index=False))

print(f"\n{'='*80}")
print("ФИНАЛЬНЫЕ МЕТРИКИ")
print(f"{'='*80}")
print(f"  Test R² (OOT): {metrics['test_r2']:.4f}  <- Основная метрика качества")
print(f"  Test RMSE:     {metrics['test_rmse']:.2f}")
print(f"  Test MAE:      {metrics['test_mae']:.2f}")
print(f"{'='*80}")

# Анализ стабильности модели
train_val_diff = abs(metrics['train_r2'] - metrics['val_r2'])
val_test_diff = abs(metrics['val_r2'] - metrics['test_r2'])

print(f"\nАНАЛИЗ СТАБИЛЬНОСТИ МОДЕЛИ:")
print(f"{'='*80}")
print(f"  |Train R² - Val R²|:  {train_val_diff:.4f}")
print(f"  |Val R² - Test R²|:   {val_test_diff:.4f}")

if val_test_diff < 0.05:
    status = "✓ Стабильная модель"
elif val_test_diff < 0.10:
    status = "⚠ Умеренная деградация"
else:
    status = "✗ Сильная деградация"

print(f"  Статус: {status}")

logger.info(f"Финальная OOT R²: {metrics['test_r2']:.4f}")

In [ ]:
# ============================================================================
# СОХРАНЕНИЕ МОДЕЛИ И АРТЕФАКТОВ
# ============================================================================

logger.info("Сохранение результатов...")

save_dir = ModelConfig.MODEL_DIR / ModelConfig.MODEL_VERSION

# Сохранение модели CatBoost
model_path = save_dir / "cltv_model_small.cbm"
cltv_model.model.save_model(str(model_path))
logger.info(f"Сохранена модель: {model_path.name}")

# Сохранение сводки метрик
metrics_path = save_dir / "model_metrics.csv"
summary_df.to_csv(metrics_path, index=False)
logger.info(f"Сохранены метрики: {metrics_path.name}")

# Сохранение feature importance
importance_path = save_dir / "feature_importance.csv"
cltv_model.feature_importance.to_csv(importance_path, index=False)
logger.info(f"Сохранена важность признаков: {importance_path.name}")

# Сохранение метаданных
metadata = {
    'model_version': ModelConfig.MODEL_VERSION,
    'training_date': datetime.now().isoformat(),
    'model_type': 'single_model',
    'target_segment': 'small (1026, 1027)',
    'features': available_features,
    'categorical_features': ModelConfig.CATEGORICAL_FEATURES,
    'model_parameters': ModelConfig.CATBOOST_PARAMS,
    'data_split': {
        'train_cutoff': ModelConfig.TRAIN_CUTOFF,
        'validation_cutoff': ModelConfig.VALIDATION_CUTOFF,
        'train_size': int(metrics['train_samples']),
        'val_size': int(metrics['val_samples']),
        'test_size': int(metrics['test_samples'])
    },
    'performance_metrics': {
        k: float(v) if isinstance(v, (int, float, np.number)) else v
        for k, v in metrics.items()
    }
}

metadata_path = save_dir / "model_metadata.json"
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)
logger.info(f"Сохранены метаданные: {metadata_path.name}")

# Сохранение объекта модели
model_object_path = save_dir / "cltv_model_object.pkl"
with open(model_object_path, 'wb') as f:
    pickle.dump(cltv_model, f)
logger.info(f"Сохранен объект модели: {model_object_path.name}")

print(f"\n{'='*80}")
print(f"РЕЗУЛЬТАТЫ СОХРАНЕНЫ")
print(f"{'='*80}")
print(f"Директория: {save_dir}")
print(f"\nСохраненные файлы:")
print(f"  - Модель CatBoost: cltv_model_small.cbm")
print(f"  - Метрики: model_metrics.csv")
print(f"  - Feature importance: feature_importance.csv")
print(f"  - Метаданные: model_metadata.json")
print(f"  - Объект модели: cltv_model_object.pkl")
print(f"{'='*80}")

In [ ]:
# ============================================================================
# ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
# ============================================================================

logger.info("Создание визуализаций...")

# График 1: Сравнение R² на разных выборках
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# График 1: Столбчатая диаграмма метрик
datasets = ['Train', 'Validation', 'OOT Test']
r2_values = [metrics['train_r2'], metrics['val_r2'], metrics['test_r2']]
colors = ['#3498db', '#2ecc71', '#e74c3c']

bars = axes[0].bar(datasets, r2_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('R² (коэффициент детерминации)', fontsize=12, fontweight='bold')
axes[0].set_title('Качество модели на разных выборках', fontsize=14, fontweight='bold', pad=15)
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].axhline(y=0.8, color='gray', linestyle='--', alpha=0.5, linewidth=1, label='Порог 0.8')
axes[0].legend()

# Добавление значений на столбцы
for bar, value in zip(bars, r2_values):
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{value:.4f}',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# График 2: Топ-15 важных признаков
top_features = cltv_model.feature_importance.head(15)
axes[1].barh(range(len(top_features)), top_features['importance'].values, 
             color='#9b59b6', alpha=0.8, edgecolor='black', linewidth=1)
axes[1].set_yticks(range(len(top_features)))
axes[1].set_yticklabels(top_features['feature'].values)
axes[1].set_xlabel('Важность признака', fontsize=12, fontweight='bold')
axes[1].set_title('Топ-15 важных признаков', fontsize=14, fontweight='bold', pad=15)
axes[1].invert_yaxis()
axes[1].grid(axis='x', alpha=0.3, linestyle='--')

plt.tight_layout()
plot_path = save_dir / 'model_performance.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
logger.info(f"Сохранен график: {plot_path.name}")
plt.show()

# График 3: Детальный анализ категориальных признаков
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# EC_SECTOR_ID распределение
sector_counts = df_train_processed['EC_SECTOR_ID'].value_counts()
axes[0, 0].bar(range(len(sector_counts)), sector_counts.values, 
               color='#3498db', alpha=0.7, edgecolor='black')
axes[0, 0].set_xticks(range(len(sector_counts)))
axes[0, 0].set_xticklabels(sector_counts.index, rotation=45, ha='right')
axes[0, 0].set_ylabel('Количество записей', fontweight='bold')
axes[0, 0].set_title('Распределение по EC_SECTOR_ID', fontweight='bold', pad=10)
axes[0, 0].grid(axis='y', alpha=0.3)

# SEGMENT_ID распределение
segment_counts = df_train_processed['SEGMENT_ID'].value_counts()
axes[0, 1].bar(range(len(segment_counts)), segment_counts.values, 
               color='#2ecc71', alpha=0.7, edgecolor='black')
axes[0, 1].set_xticks(range(len(segment_counts)))
axes[0, 1].set_xticklabels(segment_counts.index, rotation=45, ha='right')
axes[0, 1].set_ylabel('Количество записей', fontweight='bold')
axes[0, 1].set_title('Распределение по SEGMENT_ID', fontweight='bold', pad=10)
axes[0, 1].grid(axis='y', alpha=0.3)

# QUALITY_CODE распределение
quality_counts = df_train_processed['QUALITY_CODE'].value_counts()
axes[1, 0].bar(range(len(quality_counts)), quality_counts.values, 
               color='#e74c3c', alpha=0.7, edgecolor='black')
axes[1, 0].set_xticks(range(len(quality_counts)))
axes[1, 0].set_xticklabels(quality_counts.index, rotation=45, ha='right')
axes[1, 0].set_ylabel('Количество записей', fontweight='bold')
axes[1, 0].set_title('Распределение по QUALITY_CODE', fontweight='bold', pad=10)
axes[1, 0].grid(axis='y', alpha=0.3)

# SUBJECT_KIND_ID распределение
subject_counts = df_train_processed['SUBJECT_KIND_ID'].value_counts()
axes[1, 1].bar(range(len(subject_counts)), subject_counts.values, 
               color='#f39c12', alpha=0.7, edgecolor='black')
axes[1, 1].set_xticks(range(len(subject_counts)))
axes[1, 1].set_xticklabels(subject_counts.index, rotation=45, ha='right')
axes[1, 1].set_ylabel('Количество записей', fontweight='bold')
axes[1, 1].set_title('Распределение по SUBJECT_KIND_ID', fontweight='bold', pad=10)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
categorical_plot_path = save_dir / 'categorical_features_distribution.png'
plt.savefig(categorical_plot_path, dpi=150, bbox_inches='tight')
logger.info(f"Сохранен график: {categorical_plot_path.name}")
plt.show()

print(f"\nВизуализации сохранены:")
print(f"  - {plot_path}")
print(f"  - {categorical_plot_path}")